https://towardsdatascience.com/policy-gradient-reinforce-algorithm-with-baseline-e95ace11c1c4

In [1]:
import gym
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import tensorflow_probability as tfp
import os
import matplotlib.pyplot as plt

def export_plot(ys, ylabel, title, filename):
    plt.figure()
    plt.plot(range(len(ys)), ys)
    plt.xlabel("Training Episode")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.savefig(filename)
    plt.close()

In [2]:
class PolicyNet():
    def __init__(self, input_size, output_size):
        self.model = keras.Sequential(
            layers=[
                keras.Input(shape=(input_size,)),
                layers.Dense(64, activation="relu", name="relu_layer"),
                layers.Dense(output_size, activation="linear", name="linear_layer")
            ],
            name="policy")

    def action_distribution(self, observations):
        logtis = self.model(observations)
        return tfp.distributions.Categorical(logits=logtis) # softmax

    def sample_action(self, observartions):
        sample_actions = self.action_distribution(observartions).sample().numpy()
        return sample_actions

class BaselineNet():
    def __init__(self, input_size, output_size):
        self.model = keras.Sequential(
            layers=[
                keras.Input(shape=(input_size,)),
                layers.Dense(64, activation="relu", name="relu_layer"),
                layers.Dense(output_size, activation="linear", name="linear_layer")
            ],
            name="baseline")

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=3e-2)

    # predict V(s)
    def forward(self, observations):
        output = tf.squeeze(self.model(observations))
        return output

    def update(self, observartions, target):
        with tf.GradientTape() as tape:
            predictions = self.forward(observartions)
            loss = tf.keras.losses.mean_squared_error(y_true=target, y_pred=predictions)

        # backprob
        grads = tape.gradient(loss, self.model.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))

class PolicyGradient(object):
    def __init__(self, env, num_iterations=300, batch_size=2000, max_ep_len=200, output_path="../results/"):
        self.output_path = output_path
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        self.env = env
        self.observation_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.n
        self.gamma = 0.9
        self.num_iterations = num_iterations
        self.batch_size = batch_size
        self.max_ep_len = max_ep_len
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=3e-2)
        self.policy_net = PolicyNet(input_size=self.observation_dim, output_size=self.action_dim) # output action probability distribution
        self.baseline_net = BaselineNet(input_size=self.observation_dim, output_size=1) # output V(s)

    def play_games(self, env=None, num_episodes=None):
        episode = 0
        episode_rewards = []
        paths = []
        t = 0
        if not env:
            env = self.env

        while (num_episodes or t < self.batch_size):
            state = env.reset()
            states, actions, rewards = [], [], []
            episode_reward = 0

            for step in range(self.max_ep_len):
                states.append(state)
                action = self.policy_net.sample_action(np.atleast_2d(state))[0] # creating batch
                state, reward, done, _ = env.step(action)
                actions.append(action)
                rewards.append(reward)
                episode_reward += reward
                t += 1

                if (done or step == self.max_ep_len-1):
                    episode_rewards.append(episode_reward)
                    break
                if (not num_episodes) and t == self.batch_size:
                    break

            # each episode's path
            path = {"observation": np.array(states),
                    "reward": np.array(rewards),
                    "action": np.array(actions)}
            paths.append(path)
            episode += 1
            if num_episodes and episode >= num_episodes:
                break

        return paths, episode_rewards # paths is the transitions of all episodes

    def get_returns(self, paths):
        all_returns = []
        for path in paths:
            rewards = path["reward"] # a list of rewards in an episode
            returns = []
            reversed_rewards = np.flip(rewards,0)
            g_t = 0

            # discounted return of an episode
            for r in reversed_rewards:
                g_t = r + self.gamma * g_t
                returns.insert(0, g_t)

            # discounted return of all episodes
            all_returns.append(returns)

        returns = np.concatenate(all_returns)
        return returns

    def get_advantage(self, returns, observations):
        values = self.baseline_net.forward(observations).numpy() # calculate b
        advantages = returns - values # G - b

        # normalize
        advantages = (advantages-np.mean(advantages)) / np.sqrt(np.sum(advantages**2))
        return advantages

    def update_policy(self, observations, actions, advantages):
        observations = tf.convert_to_tensor(observations)
        actions = tf.convert_to_tensor(actions)
        advantages = tf.convert_to_tensor(advantages)

        with tf.GradientTape() as tape:
            log_prob = self.policy_net.action_distribution(observations).log_prob(actions)
            loss = -tf.math.reduce_mean(log_prob * tf.cast(advantages, tf.float32)) # mean of batch

        # backprob
        grads = tape.gradient(loss, self.policy_net.model.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.policy_net.model.trainable_weights))

    def train(self):
        all_total_rewards = []
        averaged_total_rewards = []

        for t in range(self.num_iterations):
            paths, total_rewards = self.play_games()
            all_total_rewards.extend(total_rewards)
            observations = np.concatenate([path["observation"] for path in paths])
            actions = np.concatenate([path["action"] for path in paths])
            returns = self.get_returns(paths)
            advantages = self.get_advantage(returns, observations)
            self.baseline_net.update(observations, returns)
            self.update_policy(observations, actions, advantages)
            avg_reward = np.mean(total_rewards)
            averaged_total_rewards.append(avg_reward)
            print("Average reward for batch {}: {:04.2f}".format(t,avg_reward))

        print("Training complete")
        np.save(self.output_path+ "rewards.npy", averaged_total_rewards)
        export_plot(averaged_total_rewards, "Reward", "CartPole-v0", self.output_path + "rewards.png")

In [27]:
env = gym.make("CartPole-v0")
model = PolicyGradient(env)
model.train()

In [29]:
# old agent play game
state = env.reset()
done = False

while not done:
    action = model.policy_net.sample_action(np.atleast_2d(state))[0]
    state, reward, done, _ = env.step(action)
    env.render()

env.close()

In [35]:
# save old agent
PATH = "D:/RL/policy-gradient/results/h5_weights.h5"
model.policy_net.model.save_weights(PATH)

In [41]:
# new agent
PATH = "D:/RL/policy-gradient/results/h5_weights.h5"
env = gym.make("CartPole-v0")
new_policy_net = PolicyNet(input_size=env.observation_space.shape[0], output_size=env.action_space.n)
new_policy_net.model.load_weights(PATH)

In [42]:
# new agent play game
state = env.reset()
done = False

while not done:
    action = new_policy_net.sample_action(np.atleast_2d(state))[0]
    state, reward, done, _ = env.step(action)
    env.render()

env.close()